# **Evaluation de la composante orienté objet du chatbot**

## **Importation**

In [0]:
import pandas as pd
import spacy
from collections import Counter
import numpy as np
from sklearn.externals import joblib

#Importation pré-traitement
from nltk.tokenize.regexp import WordPunctTokenizer
from nltk.stem import SnowballStemmer
nlp = spacy.load('fr_core_news_md')

#Importation modèles
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

#Importation données
QA = pd.read_csv('../Data/Q_A.csv',sep=";")
test_metier = pd.read_csv('../Data/Q_A_test.csv',sep=";")
vectorizer2 = joblib.load('../Data/vectoriseur2.pkl')
classifieur2 = joblib.load('../Data/classifieur2.pkl')
vectorizer1 = joblib.load('../Data/vectoriseur1.pkl')
classifieur1 = joblib.load('../Data/classifieur1.pkl')
vectorizer_themes = joblib.load('../Data/vectorizer_themes.pkl')

In [0]:
test_autre = ['Comment ça va ?','Ou est le cinéma le plus proche ?','Quand visiter la Tour-Eiffel ?','Comment faire une tartiflette ?',
              'Comment soigner mes plantes ?', 'Quel est le programme télé ce soir ?', 'Manu est il beau ?', 'Quel est le numéro de Jean-Michel ?',
              'La piscine est elle ouverte ce week end ?','Quel est la valeur arrondie de Pi?', 'Vais je devenir riche et célèbre ?',
              'Fait t-il beau à New-York ?','Les produits laitiers sont-ils nos amis pour la vie ?','Quel est le meilleur fromage ?',
              'Combien coûte une Clio ?', 'Quels sont les meilleurs bonbons de tous les temps?','Comment traiter les poux ?',
              'Quelle est la date limite de rendu de projet ?','Es-tu amoureux ?','Franck Dubosc est-il drôle ?',
              'Je souhaite construire une table.','Ou trouver des films gratuits en streaming ?']

## **Définitions des fonctions de normalisation**

In [0]:
#Définition des fonctions de prétraitement du texte
def lemmatise_text(text):
  tw_nlp = nlp(text)
  list_lem = [token.lemma_ for token in tw_nlp]
  text_lem = ' '.join(list_lem)
  return text_lem

def stem_text(text):
  tokenizer = WordPunctTokenizer()
  stemmer = SnowballStemmer('french')
  liste_racines = [stemmer.stem(token) for token in tokenizer.tokenize(text)]
  return ' '.join(liste_racines)

def normalise(text):
  #stop words, strip accent et lowercase vont être fait automatiquement
  text = text.replace('\n','').replace('\r','').split(" ")
  text = " ".join([i for i in text if i!=""])
  lemmas = lemmatise_text(text) #lemme de notre texte
  stems = stem_text(lemmas) #stem de notre texte A VOIR
  return stems

## **Evaluation du classifieur n°1 : pertinence générale**

In [7]:
corpus_test = list(test_metier.Questions[:22]) + test_autre
themes_test = ['metier']*22 + ['autres']*22
corpus_norm = pd.Series(corpus_test).apply(normalise)
corpus_vect = vectorizer1.transform(corpus_norm)
themes_pred = classifieur1.predict(corpus_vect)
themes_test_bin = [1 if t=='metier' else 0 for t in themes_test]
themes_pred_bin = [1 if t=='metier' else 0 for t in themes_pred]
print("Accuracy : {} \nPrecision : {} \nRecall : {} \nF1 Score : {}".format(accuracy_score(themes_test, themes_pred),
                                                                            precision_score(themes_test_bin, themes_pred_bin),
                                                                            recall_score(themes_test_bin, themes_pred_bin),
                                                                            f1_score(themes_test_bin, themes_pred_bin)))
Counter(themes_pred),Counter(themes_test)
pd.DataFrame(confusion_matrix(themes_test,themes_pred),index=['autres','metier'],columns=['autres','metier'])

Accuracy : 0.6136363636363636 
Precision : 0.5641025641025641 
Recall : 1.0 
F1 Score : 0.7213114754098361


,autres,metier
autres,5,17
metier,0,22


## **Evaluation du classifieur n°2 : thématique**

In [10]:
corpus_test = test_metier.Questions
themes_test = test_metier.Themes
corpus_norm = pd.Series(corpus_test).apply(normalise)
corpus_vect = vectorizer2.transform(corpus_norm)
themes_pred = classifieur2.predict(corpus_vect)
print("Accuracy :{}".format(accuracy_score(themes_test, themes_pred)))
Counter(themes_test)
Counter(themes_pred)
themes = ['Administratif/Argent','Bagages','Hébergement','Santé/Sécurité',
          'Transport','Trek/Visite/Guide','Vivre au Pérou/Culture','Voyage']
pd.DataFrame(confusion_matrix(themes_test, themes_pred),index=themes,columns=themes)

Accuracy :0.8775510204081632


,Administratif/Argent,Bagages,Hébergement,Santé/Sécurité,Transport,Trek/Visite/Guide,Vivre au Pérou/Culture,Voyage
Administratif/Argent,18,0,0,1,0,0,0,1
Bagages,0,6,0,0,0,0,0,1
Hébergement,0,0,7,0,0,0,0,0
Santé/Sécurité,0,0,0,19,1,0,1,0
Transport,0,0,0,0,15,0,0,0
Trek/Visite/Guide,1,0,0,0,2,6,0,0
Vivre au Pérou/Culture,0,0,0,1,0,0,7,0
Voyage,0,0,1,2,0,0,0,8


## **Evaluation de la réponse du ChatBot**

In [12]:
n = len(test_metier)
prediction_rep = [0]*n

for i in range(n):
  theme = test_metier.Themes.iloc[i]
  user_input = test_metier.Questions.iloc[i]
  vect, dtm, ind = vectorizer_themes[theme]
  user_norm = pd.Series([user_input]).apply(normalise)
  user_vector = vect.transform(user_norm)
  query_corpus_sim = np.squeeze(cosine_similarity(dtm, user_vector))
  idx_most_sim = np.argmax(query_corpus_sim)
  best = ind[idx_most_sim%len(ind)]
  prediction_rep[i] = QA.iloc[best].Answers

print('Accuracy : {}'.format(accuracy_score(test_metier.Answers, prediction_rep)))

Accuracy : 0.6836734693877551
